### 交叉验证

In [2]:
import numpy as np
from sklearn import datasets

In [3]:
digits = datasets.load_digits() # 用手写识别数据
X = digits.data
y = digits.target

### 测试train_test_split
测试多组train_test_split得到的模型

In [4]:
# 为什么不是交叉验证啊？
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666) # 也就是测试数据集占据40%

In [6]:
from sklearn.neighbors import KNeighborsClassifier

# 变量初始化可以中途添加，比如下面的
best_score, best_p, best_k = 0, 0, 0
# 使用knn
# 训练模型调节参数，调节参数就是使用网格搜索的方法：这个嵌套for循环是网格搜索吗？忘了
for k in range(2, 11):
    for p in range(1, 6): # p是曼哈顿距离阶数
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score, best_p, best_k = score, p, k

print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 3
Best P = 4
Best Score = 0.9860917941585535


### 使用交叉验证
通常情况下，我会更相信通过交叉验证得到的这组参数  
1. 因为很可能模型只是过拟合了我们随机分出来的测试数据集而已  
2. Best Score 交叉验证过程中通常不会过拟合某一组的测试数据，所以平均来讲分数稍微低一些
3. 用新的模型参数作用于train_test_split，此时就不要用train_validation_test_split了
4. 交叉验证过程中没有使用测试数据集哦，这个计算出来的准确率相对来说是值得相信的

In [7]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([0.98895028, 0.97777778, 0.96629213])

In [9]:
from sklearn.neighbors import KNeighborsClassifier

# 变量初始化可以中途添加，比如下面的
best_score, best_p, best_k = 0, 0, 0
# 使用knn
# 训练模型调节参数，调节参数就是使用网格搜索的方法：这个嵌套for循环是网格搜索吗？忘了
for k in range(2, 11):
    for p in range(1, 6): # p是曼哈顿距离阶数
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        # knn_clf.fit(X_train, y_train)
        # score = knn_clf.score(X_test, y_test)
        scores = cross_val_score(knn_clf, X_train, y_train) # X_test，y_test没有参数交叉验证的计算过程
        score = np.mean(scores)
        if score > best_score:
            best_score, best_p, best_k = score, p, k

print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 2
Best P = 2
Best Score = 0.9823599874006478


In [10]:
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)

In [11]:
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

### 回顾网格搜索

In [12]:
# weights超参数固定，只有一种选择
from sklearn.model_selection import GridSearchCV

# 参数列表传递给对象
param_grid = [
    # 字典
    {
        "weights": ['distance'],
        "n_neighbors": [i for i in range(2, 11)],
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train) # 生成3个模型，对每组参数训练三次求平均

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  2.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=5,
           weights='distance'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [13]:
grid_search.best_score_

0.9823747680890538

In [14]:
# 我们找到的最佳参数是什么？
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [15]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975

In [16]:
# 训练5个模型，每个模型相应的分数是多少
cross_val_score(knn_clf, X_train, y_train, cv=5)

array([0.99543379, 0.96803653, 0.98148148, 0.96261682, 0.97619048])

In [17]:
GridSearchCV(knn_clf, param_grid, verbose=1, cv=5)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=5,
           weights='distance'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)